<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [657]:
# Устанавливаем библиотеки BeautifulSoup и lxml

#!pip install beautifulsoup4 
#!pip install lxml



import pandas as pd
import psycopg2
import warnings
import requests
import plotly.express as px
import os
from bs4 import BeautifulSoup

warnings.filterwarnings('ignore')

path = os.path.dirname(os.getcwd())
path = path.replace('\\', '/')
print(path)


c:/Users/Yaroslav/Desktop/Python_repos/SQL__StudyPrject2


In [658]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [659]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [660]:
# текст запроса
query_3_1 = f''' 
-- из таблицы с вакансиями выводим кол-во строк
select
    count(*) as N_row 
from
    vacancies

'''

In [661]:
vacancies_df = pd.read_sql_query(query_3_1, connection)
vacancies_df

n_row
0  49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [662]:
query_3_2 = '''
-- из таблицы с работодателями выводим кол-во уникальных названий 
select
    count(distinct name) as N_employers
    
from
    employers
'''

In [663]:
employers_df = pd.read_sql_query(query_3_2, connection)
employers_df

n_employers
0        23175

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [664]:
query_3_3 = ''' 
-- из таблицы с регионами выводим кол-во уникальных названий 
select
    count(distinct name) as N_areas
    
from
    areas   
'''

In [665]:
areas_df = pd.read_sql_query(query_3_3, connection)
areas_df

n_areas
0     1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [666]:
query_3_4 =''' 
-- из таблицы со сферами деятельности выводим количество уникальных названий
select
    count(distinct name) as N_areas
    
from
    industries       
    
'''

In [667]:
idustries_df = pd.read_sql_query(query_3_4, connection)
idustries_df

n_areas
0      294

***

1. В таблице vacancies, которая отражает общее количество вакансий, 49197 строк. <br>
Кличество уникальных названий вакансий составило 21223
2. В таблице employers, которая отражает общее количество работодателей, 23501 строка. <br>
Кличество уникальных названий работадателя составило 23175
3. В таблице areas, которая отражает общее количество регионов, 1362 строки. <br>
Все названия (names) уникальны.

4. В таблице industries которая отражает количество сфер деятельности, 294 строки <br>
Все названия (names) уникальны.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [668]:
query_4_1 = ''' 

-- к таблице с вакансиями присоединяем таблицу с городами по area_id
-- выводим название городов и посчитываем кол-во строк
select
    a.name,
    count(*) N_vacs
    
from
    vacancies v
    join areas a
    on v.area_id = a.id

-- группируем по названию городов
group by a.name

-- сортируем по кол-ву строк в нисходящем порядке 
order by count(*) desc

-- выводим только первые 10 элементов
limit 10

'''

In [669]:
VacAreaJoin = pd.read_sql_query(query_4_1, connection)
VacAreaJoin

name  n_vacs
0           Москва    5333
1  Санкт-Петербург    2851
2            Минск    2112
3      Новосибирск    2006
4           Алматы    1892
5     Екатеринбург    1698
6  Нижний Новгород    1670
7           Казань    1415
8        Краснодар    1301
9           Самара    1144

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [670]:
query_4_2 = ''' 
-- посчитываем кол-во строк из таблицы с вакансиями
select
    count(*) as N_slry_non_null
    
from 
    vacancies
    
-- выбираем только те строки, где хотя бы одно из полей с ЗП заполнено   
where 
    (salary_from is not null) or (salary_to is not null)
    
'''


In [671]:
slry_not_null = pd.read_sql_query(query_4_2, connection)
slry_not_null

n_slry_non_null
0            24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [672]:
query_4_3 = ''' 
-- выводим среднюю ЗП от и среднюю ЗП до из таблицы с вакансиями
select
    round(avg(salary_from)) as avg_slry_from,
    round(avg(salary_to)) as avg_slry_to
    
from 
    vacancies

'''

In [673]:
avg_slry = pd.read_sql_query(query_4_3, connection)
avg_slry

avg_slry_from  avg_slry_to
0        71065.0     110537.0

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. <br> Результат отсортируйте по убыванию количества.


In [674]:
query_4_4 = ''' 
-- из таблицы с вакансиями выводим режим работы и занятость 
-- подсчитываем кол-во строк 
select
    schedule,
    employment,
    count(*) N_vac
    
from
    vacancies

-- группируем данные по режиму работы и занятости 
group by 
    schedule, employment

-- сортируем по количеству вакансий в убывающем порядке
order by count(*) desc
'''

In [675]:
sch_empl_count = pd.read_sql_query(query_4_4, connection)
sch_empl_count

schedule           employment  n_vac
0        Полный день     Полная занятость  35367
1   Удаленная работа     Полная занятость   7802
2      Гибкий график     Полная занятость   1593
3   Удаленная работа  Частичная занятость   1312
4     Сменный график     Полная занятость    940
5        Полный день           Стажировка    569
6     Вахтовый метод     Полная занятость    367
7        Полный день  Частичная занятость    347
8      Гибкий график  Частичная занятость    312
9        Полный день     Проектная работа    141
10  Удаленная работа     Проектная работа    133
11     Гибкий график           Стажировка    116
12    Сменный график  Частичная занятость    101
13  Удаленная работа           Стажировка     64
14     Гибкий график     Проектная работа     18
15    Сменный график           Стажировка     12
16    Вахтовый метод     Проектная работа      2
17    Сменный график     Проектная работа      1

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [676]:
query_4_5 = ''' 
-- выводим опыт работы и подсчитваем кол-во строк из таблицы с вакансиями 
select
    experience,
    count(*) as N_vac
    
from
    vacancies

-- группируем по опыту работы    
group by experience

-- сортируем по кол-ву вакансий в восходящем порядке
order by count(*) asc

'''

In [677]:
count_by_exp = pd.read_sql_query(query_4_5, connection)
count_by_exp

experience  n_vac
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152

***

1. Наибольшее количество вакансий представлено в Москве и Санкт-Петербурге. Также, в топе столицы сосдних государств и города-миллионники.

2. Только у 24073 соискателей из 49197 (чуть менее половины) указана хотя бы одна граница зарплатной вилки

3. В среднем, нижняя граница зарплатной вилки составила 71к рублей, верхней - 110к рублей

4. Соискатели, преимущественно, предпочитают полную занятость и полный день (~35к). На втором месте удаленная работа и полная занятость (~8к).<br>
На все остальные сочетания графика работы и занятости приходится ~6к
5. Подавляющее бльшинство соискателей (~26к) имеет опыть работы до 3-х лет. От 3-х до 6-ти лет у ~14к соискателей. <br>
~7к соискателей не имеют опыта работы. Опыт работы более 6 лет имеют всего 1337 соискателей.

ЗАМЕНИТЬ СОИСКАТЕЛЕЙ НА ВАКАНСИИ


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [678]:
query_5_1 = ''' 

-- к таблице с вакансиями присоединяем работодателей по employer_id
-- подсчитываем количество строк
select
    e.name,
    count(*) N_vacs
    
from
    vacancies v
    join employers e
    on v.employer_id = e.id

-- группируем по названиям работодателей
group by
    e.name  

-- сортируем по количеству строк в убывающем порядке
order by count(*) desc

'''

In [679]:
# запрос записываем в переменную и выводим первые 5 строк
vac_count_by_empl = pd.read_sql_query(query_5_1, connection)
vac_count_by_empl.head(5)

name  n_vacs
0         Яндекс    1933
1     Ростелеком     491
2       Тинькофф     444
3           СБЕР     428
4  Газпром нефть     331

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём. <br>
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [680]:
query_5_2 =''' 

-- к таблице с городами присоединяем вакансии, а также таблицу с работодателями
-- выводим названия городов и посчитываем количество работодателей
select
    a.name,
    count(e.name) N_employers
   
from
    areas a
    left join vacancies v 
    on v.area_id = a.id
    
    join employers e 
    on a.id = e.area

-- выбираем те вакансии, у которых отсутсвует id
where 
    v.id is null

-- группируем по названию города 
group by 
    a.name

-- сортируем по количеству работодателей в убывающем порядке
order by 
    count(e.name) desc
    
'''

In [681]:
Null_vac = pd.read_sql_query(query_5_2, connection)
Null_vac

name  n_employers
0                                       Россия          410
1                                    Казахстан          207
2                           Московская область           75
3                           Краснодарский край           19
4                                     Беларусь           18
..                                         ...          ...
181                                  Миллерово            1
182  Михайловка (Уфимский район, Башкортостан)            1
183                                    Можайск            1
184                                Морозовск\n            1
185                                     Несвиж            1

[186 rows x 2 columns]

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [682]:
query_5_3 = ''' 

-- к таблице с вакансиями присоединяем работодателей по employer_id 
-- и города по area_id. Выводим название работодателей,
-- а также посчитываем колчество уникальных названий городов
select
    e.name as Employer_name,
    count(distinct a.name) as N_regions_with_vac
    
from
    vacancies v
    join employers e
    on v.employer_id = e.id
    
    join areas a
    on v.area_id = a.id

-- группируем по названию работодателя
group by 
    e.name
   
-- сортируем по кол-ву городов в убывающем порядке 
order by 
    count(distinct a.name) desc
'''


In [683]:
N_region_by_Employer = pd.read_sql_query(query_5_3, connection)
N_region_by_Employer

employer_name  n_regions_with_vac
0                      Яндекс                 181
1                  Ростелеком                 152
2                  Спецремонт                 116
3      Поляков Денис Иванович                  88
4                   ООО ЕФИН                   71
...                       ...                 ...
14761                  UniSol                   1
14762            UNISTORY LLC                   1
14763                   UNIT6                   1
14764     United Distribution                   1
14765              UNITEL LLC                   1

[14766 rows x 2 columns]

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [684]:
query_5_4 = ''' 

-- к таблице с работадателями присоединяем таблицу  employers_industries
-- и посчитываем количество строк
select 
    count(e.name) N_empl_with_no_indstr
    
from
    employers e
    left join employers_industries ei 
    on ei.employer_id = e.id

-- выбираем данные, где отстутсвуют industry_id
where industry_id is null

'''


In [685]:
N_Empl_null_Indstr = pd.read_sql_query(query_5_4, connection)
N_Empl_null_Indstr

n_empl_with_no_indstr
0                   8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [686]:
query_5_5 = ''' 

-- к таблице employers_industries присоединяем работодателей по employer_id
-- группируем по имени работодателя и посчитываем кол-во строк 
select 
    e.name,
    count(ei.industry_id)
    
from
    employers_industries ei
    join employers e
    on ei.employer_id = e.id 

-- группируем по имени работодателя      
group by 
    e.name

-- выбираем тех работодателей, у которых 4 направления деятельности 
having 
    count(ei.industry_id) = 4

-- сортируем по имени и выводим третий элемент
order by 
    e.name
    
offset 2
limit 1
'''

In [687]:
Indstr_by_Empl = pd.read_sql_query(query_5_5, connection)
Indstr_by_Empl

name  count
0  2ГИС      4

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [688]:
query_5_6 = ''' 

-- к таблице employers_industries присоединяем таблицу с работадателями 
-- по employer_id и таблицу с видами деятельности по industry_id 
-- выводим названия видов деятельности и посчитываем кол-во строк

select 
    i.name,
    count (*) N_emplyers
    
from
    employers_industries ei
    join employers e
    on ei.employer_id = e.id 
    
    join industries i
    on ei.industry_id = i.id

-- группируем по названию родов деятельности
group by 
    i.name

-- выбираем названия, которые соотвествуют 'Разработка программного обеспечения'
having 
    i.name = 'Разработка программного обеспечения'
'''


In [689]:
N_dev_Empl = pd.read_sql_query(query_5_6, connection)
N_dev_Empl

name  n_emplyers
0  Разработка программного обеспечения        3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [690]:
# Выделяем таблицу с названиями городов-миллионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_="standard sortable")


# Создаем список с названиями городов-миллионников
df = pd.read_html(str(table))
df = pd.concat(df) 
city_names = list(df.loc[:,'Город'])

In [691]:
query_5_7 = ''' 

-- таблице с вакансиями присоединяем таблицу с работадателями по employer_id и
-- таблицу с городами по area_id и выводим названия городов и их id для посчета
select 
    a.name as City,
    v.id as vacancy_cnt
    
from 
    vacancies v
    join employers e
    on v.employer_id = e.id
    
    join areas a
    on v.area_id = a.id

-- оставляем только вакансии от "Яндекс"
where e.name = 'Яндекс' 
'''

In [692]:
# Записываем результат запроса в датафрейм
YaVac_by_City = pd.read_sql_query(query_5_7, connection)


# Выбираем только данные из городов-миллионников
mask = YaVac_by_City.loc[:, 'city'].apply(lambda x: x in city_names)
YaVac_by_City = YaVac_by_City[mask]


# Группируем по переменной 'City', подссчитываем кол-во вакансий
YaVac_by_City = \
    YaVac_by_City.groupby('city').count()\
        .sort_values('vacancy_cnt', ascending=False)


# Добавляем строрку 'Total' с суммой по солбцу с вакансиями
YaVac_by_City.loc['Total'] = sum(YaVac_by_City['vacancy_cnt'])
display(YaVac_by_City)


vacancy_cnt
city                        
Москва                    54
Санкт-Петербург           42
Екатеринбург              39
Нижний Новгород           36
Новосибирск               35
Воронеж                   32
Краснодар                 30
Самара                    26
Уфа                       26
Казань                    25
Пермь                     25
Ростов-на-Дону            25
Волгоград                 24
Красноярск                23
Челябинск                 22
Омск                      21
Total                    485

***

1.	Наибольшее количество вакансий (1933) предлагает компания **Яндекс**. В пятерку лидеров также входят **Ростелеком**, **Тинькофф**, **СБЕР** и **Газпром нефть**.

2.	Компании **Яндекс**, **Ростелеком** и **Спецремонт** входят в тройку лидеров по количеству регионов, в которых они предлагают вакансии. 

3.	Компания **Яндекс** предлагает **485** вакансий в городах-миллионниках. Наибольшее количество вакансий приходится на **Москву**, **Санкт-Петербург**, **Екатеринбург**, **Нижний Новгород** и **Новосибирск**.

4.	Среди всех работодателей, у **8419** не указан род деятельности. У **3553** компаний в качестве рода деятельности указана разработка программного обеспеченья.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [693]:
query_6_1 = ''' 

-- из таблицы с вакансиями выбираем все названия, 
-- в которых есть слова "data" или "данн"
select
    name
    
from
    vacancies
    
where 
    lower(name) like '%data%' 
    or 
    lower(name) like '%данн%'
'''

Data_Vac = pd.read_sql_query(query_6_1, connection)
Data_Vac

name
0     Аналитик хранилищ данных и BI-систем/Аналитик ...
1                              Data-analysts (удаленно)
2     Стажер Data Engineer/Data Аналитик в команду Б...
3                    Старший специалист хранилищ данных
4                      Администратор баз данных и сетей
...                                                 ...
1766        Программист Delphi / разработчик баз данных
1767                  Разработчик баз данных (Big Data)
1768                                    Аналитик данных
1769              Data аналитик /Аналитик BI (удаленно)
1770                                   DataOps Engineer

[1771 rows x 1 columns]

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [694]:
query_6_2 = ''' 

-- из таблицы с вакансиями выбираем название, опыт и занятость
select
    name,
    experience,
    employment
    
from
    vacancies
    
where 

    -- фильтруем данные в соответсвии с условием 
    (lower(name) like 'ml%'or
     lower(name) like '% ml%'or
     lower(name) like '%-ml%'or
     lower(name) like '%/ml%'or
     lower(name) like '%(ml%'or
     
        
     lower(name) like '%data scientist%' or 
     lower(name) like '%data science%' or 
     lower(name) like '%исследователь данных%' or
     lower(name) like '%machine learning%' or 
     lower(name) like '%машинн%обучен%')
     
    and
    
    -- выбираем данные, где либо есть слово junior в названии, 
    -- либо в занятости указана "Стажировка",
    -- либо в опыт работы - "БЕза опыта"
    (lower(name) like '%junior%' or 
     employment like 'Cтажировка' or
     experience like 'Нет опыта')
'''


In [695]:
Data_Vac_Jun = pd.read_sql_query(query_6_2, connection)
display(Data_Vac_Jun)

name          experience  \
0                             Junior+ CV/ML developer  От 1 года до 3 лет   
1                            Data Scientist (Junior+)  От 1 года до 3 лет   
2                      Junior Data Scientist (Стажер)           Нет опыта   
3      Математик, Junior/Middle/Senior Data Scientist  От 1 года до 3 лет   
4   Data Scientist/Machine Learning Engineer (Deve...           Нет опыта   
5                               Junior Data Scientist           Нет опыта   
6                             Data Scientist (Junior)           Нет опыта   
7                        Intern/Junior Data Scientist           Нет опыта   
8                          Junior Data Scientist (VK)  От 1 года до 3 лет   
9                        Junior/Middle Data Scientist  От 1 года до 3 лет   
10          Data Scientist (Computer Vision) - Junior           Нет опыта   
11  Исследователь в области машинного обучения и м...           Нет опыта   
12                             Data Scientist, Junior           Нет опыта   
13                    Data Scientist (junior+/middle)  От 1 года до 3 лет   
14  Junior Data Scientist Управления модельных рис...  От 1 года до 3 лет   
15  Математик-аналитик (data scientist / data anal...           Нет опыта   
16                              Junior Data Scientist  От 1 года до 3 лет   
17                 Research Engineer / Data Scientist           Нет опыта   
18  Стажер направления Машинное обучение и анализ ...           Нет опыта   
19                              Junior Data Scientist  От 1 года до 3 лет   
20  Стажёр Data Scientist / Группа сервисов Антифр...           Нет опыта   
21             Junior Data Scientist (Promo Forecast)  От 1 года до 3 лет   
22          Senior Research Engineer / Data Scientist           Нет опыта   
23                              Ведущий AI/ML инженер           Нет опыта   
24        Data scientist (Junior/Middle) в консалтинг           Нет опыта   
25  Junior Data Scientist (математик-аналитик) уда...  От 1 года до 3 лет   
26  Data Scientist/Machine Learning Engineer (Deve...           Нет опыта   
27  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
28                              Data Scientist в Сбер           Нет опыта   
29  Data Scientist/Machine Learning Engineer (Deve...           Нет опыта   
30  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
31  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
32  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
33  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
34  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
35  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
36  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
37  Junior Data Scientist (Валидация скоринговых м...  От 1 года до 3 лет   
38            Риск - Аналитик (Junior Data scientist)  От 1 года до 3 лет   
39  Программист Python/ Data Scientist (junior / m...           Нет опыта   
40            Data Scientist/ML engineer (Python/C++)           Нет опыта   
41       Риск-аналитик (ML Engineer / Data Scientist)           Нет опыта   
42       Junior Data scientist/ Младший риск-аналитик  От 1 года до 3 лет   
43                         Senior Data Scientist (AI)           Нет опыта   
44             Junior Data Scientist for NLP (Python)           Нет опыта   
45          Junior Research Engineer / Data Scientist  От 1 года до 3 лет   
46                              Ведущий AI/ML инженер           Нет опыта   
47              Python developer / Junior ML engineer           Нет опыта   
48                                     Data Scientist           Нет опыта   
49          Data Scientist (Computer Vision) / Junior           Нет опыта   
50                             Data Scientist, Junior           Нет опыта   

             employment  
0      Полная заня

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [696]:
query_6_3 = ''' 

-- из таблицы с вакансиями выбираем название и ключевые навыки 
select 
    name,
    key_skills

from 
    vacancies

where 
    -- фильтруем данные в соответсвии с условием 
    ((name ilike '%ML%' and name not ilike '%HTML%') or        
    (lower(name) like '%data scientist%') or 
    (lower(name) like '%data science%') or
    (lower(name) like '%исследователь данных%') or
    (lower(name) like '%machine learning%') or
    (lower(name) like '%машинн%обучен%') 
    
    AND 
    
    -- исключаем данные, где key_skills отсутсвуют 
    (key_skills IS NOT NULL))

    AND
    
    -- выбираем данные, где key_skills включают SQL или Postgres
    (key_skills like '%SQL%' or key_skills ilike '%postgres%')

order by name
    
'''


In [697]:
Data_Vac_SQL = pd.read_sql_query(query_6_3, connection)
Data_Vac_SQL

name  \
0                               Analyst/Data Scientist   
1    B2B Data Scientist (Эксперт по моделированию и...   
2      Backend- разработчик (Middle/Senior-TeamLeader)   
3    Chief Data Science (Цифровой кредитный монитор...   
4    Commercial Data Scientist / Эксперт по наукоем...   
..                                                 ...   
224  Специалист по разработке систем технического з...   
225  Специалист по углубленной аналитике данных (Da...   
226  Стажер направления Машинное обучение и анализ ...   
227  Стажёр Data Scientist / Группа сервисов Антифр...   
228                    Старший аналитик (Data Science)   

                                            key_skills  
0    Английский язык\tНавыки презентации\tСбор и ан...  
1        Python\tSQL\tPostgreSQL\tGit\tАнглийский язык  
2    JavaScript\tMySQL\tОбучение и развитие\tУправл...  
3                 Python\tSpark\tHadoop\tSQL\tBig Data  
4    Python\tData Analysis\tMachine Learning\tTenso...  
..                                                 ...  
224                 Python\tLinux\tMongoDB\tPostgreSQL  
225  SPSS\tСтатистический анализ\tStatistica\tАнали...  
226                                 SQL\tOracle Pl/SQL  
227  Python\tSQL\tPostgreSQL\tML\tMachine Learning\...  
228  Data Mining\tMS SQL Server\tOlap (online analy...  

[229 rows x 2 columns]

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [698]:
query_6_4 = ''' 

-- из таблицы с вакансиями выбираем название и ключевые навыки 
select 
    name,
    key_skills

from 
    vacancies

where 

    -- фильтруем данные в соответсвии с условием 
    ((name ilike '%ML%' and name not ilike '%HTML%') or        
    (lower(name) like '%data scientist%') or 
    (lower(name) like '%data science%') or
    (lower(name) like '%исследователь данных%') or
    (lower(name) like '%machine learning%') or
    (lower(name) like '%машинн%обучен%') )
    
    AND 
    
    -- исключаем данные, где key_skills отсутсвуют 
    (key_skills IS NOT NULL)

    AND
    
    -- выбираем данные, где key_skills включают Python 
    (key_skills ilike '%python%')

order by name
    
'''

In [699]:
Data_Vac_Python = pd.read_sql_query(query_6_4, connection)
Data_Vac_Python

name  \
0                                AI/ML Senior Engineer   
1                                AI/ML Senior Engineer   
2                               Analyst/Data Scientist   
3    B2B Data Scientist (Эксперт по моделированию и...   
4    Chief Data Science (Цифровой кредитный монитор...   
..                                                 ...   
352  Специалист по углубленной аналитике данных (Da...   
353  Стажёр Data Scientist / Группа сервисов Антифр...   
354                  Старший разработчик (ML engineer)   
355  Эксперт по машинному обучению и искусственному...   
356  Эксперт по машинному обучению и искусственному...   

                                            key_skills  
0                       Python\tFlask\tLinux\tAWS\tGit  
1                       Python\tFlask\tLinux\tAWS\tGit  
2    Английский язык\tНавыки презентации\tСбор и ан...  
3        Python\tSQL\tPostgreSQL\tGit\tАнглийский язык  
4                 Python\tSpark\tHadoop\tSQL\tBig Data  
..                                                 ...  
352  SPSS\tСтатистический анализ\tStatistica\tАнали...  
353  Python\tSQL\tPostgreSQL\tML\tMachine Learning\...  
354  Python\tLinux\tPyTorch\tMachine Learning\tData...  
355  Python\tМатематическая статистика\tМатематичес...  
356  Python\tМатематическая статистика\tМатематичес...  

[357 rows x 2 columns]

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [700]:
query_6_5 = ''' 

-- из таблицы с вакансиями выбираем название, навыки и подчитываем кол-во навыков
select 
    v.name, 
    v.key_skills,
    
    -- для подсчета кол-ва навыков от строки key_skills 
    -- отнимаем строку key_skills, где все разделители удалены и прибавляем единицу
    length(v.key_skills) - length(replace(v.key_skills, CHR(9), ''))+1 n_skills

from 
    vacancies v

where 
   -- фильтруем данные в соответсвии с условием 
    (
    (name ILIKE '%data scientist%') 
    OR (name ILIKE '%data science%') 
    OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')  
    )
    
    -- исключаем данные, где key_skills отсутсвуют 
    AND (key_skills IS NOT NULL)
    
    order by v.name
'''


In [701]:
# записываем запрос в перемменную и расчитываем средннее по столбику 'n_skills'
DS_Vacs = pd.read_sql_query(query_6_5, connection)
Avg_vacs = round(DS_Vacs['n_skills'].mean(), 2)

print(f'Среднее количество запрашиваемых навыков составляет: {Avg_vacs}')

Среднее количество запрашиваемых навыков составляет: 6.55


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [702]:
add_query_3 = ''' 

-- присоединяем к таблице с вакансиями названия городов
-- выводим название вакансии, название города, опыт работы, 
-- занятость и расписание 
select 
    v.experience,
    coalesce(salary_from, 0) + coalesce(salary_to, 0)   Avg_slry
    
from 
    vacancies v
   

where 
     -- выбираем данные, связанные с DS и ML
     (v.name ilike 'ml%'or
     v.name ilike '% ml%'or
     v.name ilike '%-ml%'or
     v.name ilike '%/ml%'or
     v.name ilike '%(ml%'or
     
        
     v.name ilike '%data scientist%' or 
     v.name ilike '%data science%' or 
     v.name ilike '%исследователь данных%' or
     v.name ilike '%machine learning%' or 
     v.name ilike '%машинн%обучен%')
      
     
     
     AND 
     
    (coalesce(salary_from, 0) = 0 or coalesce(salary_to, 0) = 0 )   
    
     AND 
    
    coalesce(salary_from, 0) + coalesce(salary_to, 0) != 0
    
union all


select 
    v.experience,
    (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2   Avg_slry
    
from 
    vacancies v
   

where 
     -- выбираем данные, связанные с DS и ML
     (v.name ilike 'ml%'or
     v.name ilike '% ml%'or
     v.name ilike '%-ml%'or
     v.name ilike '%/ml%'or
     v.name ilike '%(ml%'or
     
        
     v.name ilike '%data scientist%' or 
     v.name ilike '%data science%' or 
     v.name ilike '%исследователь данных%' or
     v.name ilike '%machine learning%' or 
     v.name ilike '%машинн%обучен%')
     
     AND 
     
    (coalesce(salary_from, 0)!=0 and coalesce(salary_to, 0) !=0)


'''


In [703]:
DS_Avg_Slry = pd.read_sql_query(add_query_3, connection)
round(DS_Avg_Slry.groupby('experience', as_index=False)['avg_slry'].mean())

experience  avg_slry
0           Нет опыта   74643.0
1  От 1 года до 3 лет  142626.0
2       От 3 до 6 лет  243115.0

***

1.	В приведённом датасете к анализу данных относится **1771** вакансия. При этом **461** вакансия относится <br>
к *Data science* или *ML*. **51** вакансия подходит для новичков и не требует опыта работы.

2.	В **229** вакансиях в качестве ключевого навыка указан *SQL* или *PostgreSQL*. *Python* является ключевым <br>
навыком в **357** вакансиях. В среднем, количество запрашиваемых навыков составило **6.5**.

3.	Средняя зарплата в вакансиях, не требующих опыта работы, составила **74к**. При опыте работы от *1-го <br>
года до 3-х лет*, ЗП увеличивается практически вдвое и составляет в среднем **144к**. Удивительно, что средняя <br>
ЗП при опыте работы *более 6-ти лет* ниже, чем при опыте работы *от 3-х до 6-ти лет*. По-видимому, это<br> 
связано с тем, что в вакансиях *от 6-ти лет* указывают только нижнюю границу зарплатной вилки.


# Общий вывод по проекту

In [704]:
add_query_1 = ''' 

-- присоединяем к таблице с вакансиями названия городов
-- выводим название вакансии, название города, опыт работы, 
-- занятость и расписание 
select 
    v.name vac_name,
    a.name city,
    experience,
    employment,
    schedule
     
from 
    vacancies v 
    join areas a 
    on v.area_id=a.id

where 

-- выбираем данные, связанные с DS и ML
     v.name ilike 'ml%'or
     v.name ilike '% ml%'or
     v.name ilike '%-ml%'or
     v.name ilike '%/ml%'or
     v.name ilike '%(ml%'or
     
        
     v.name ilike '%data scientist%' or 
     v.name ilike '%data science%' or 
     v.name ilike '%исследователь данных%' or
     v.name ilike '%machine learning%' or 
     v.name ilike '%машинн%обучен%'
'''

In [706]:
# записываем SQL запрос в датафрейм
add_df_1 = pd.read_sql_query(add_query_1, connection)

# выбираем только те данные, которые относятся к городам-миллионникам
mask = add_df_1.loc[:, 'city'].apply(lambda x: x in city_names)
add_df_1 = add_df_1[mask]

# группируем данные по опыту работы и городу; подсчитываем кол-во вакансий
exp_by_city = add_df_1.groupby(['experience', 'city'], \
    as_index=False)['vac_name'].count().sort_values(['experience', 'city'])

# создаем Pivot-таблицу
exp_by_city
pivot = pd.pivot_table(
    data = exp_by_city,
    index = 'experience',
    columns = 'city',
    values = 'vac_name'
)

# переставляем местами строки и колонки 
# (удаляем Красноярск из списка городов т.к. там нету удовлетворяющих нас вакансий)
pivot = pivot.reindex(['Нет опыта', 'От 1 года до 3 лет',
                       'От 3 до 6 лет', 'Более 6 лет'])

city_names_cp = city_names.copy()
city_names_cp.remove('Красноярск')

pivot = pivot[city_names_cp]


# выводим полученные данные в виде тепловой карты
Vac_by_CityExp = px.imshow(pivot,
                           color_continuous_scale=['whitesmoke', 'darkred'],
                           labels=dict(x='', y ='Опыт работы'),                   
                           width=1200,
                           title = 'Зависимость количества вакансий от опыта работы и города <br>\
(данные представлены только по городам-миллионникам)')

Vac_by_CityExp.show()
Vac_by_CityExp.write_html(path + '/HTMLs/Vac_by_CityExp.html')
Vac_by_CityExp.write_image(path + '/PNGs/Vac_by_CityExp.png', scale=6.5)

* На данной тепловой карте показано, что наибольшее количество работодателей ищут сотрудников с опытом работы *1 год и более*.<br>
При этом сотрудников *без опыта работы* ищут немного чаще, чем и сотрудников со значительным *опытом работы более 6 лет*. 

* Основное количестов вакансий наблюдается в **Москве** и **Санк-Петербурге**, при этом в **Москве** требуется гораздо больше (до трех раз) сотрудников в каждой категории.<br>
Только в этих двух городах ищут сотрудников *с опытом работы более 6-ти лет*.
* В пятерку лидеров по количеству вакансий также входят **Новосибирск**, **Казань** и **Нижний Новгород**, хотя и со значительным отрывом.

In [707]:
add_query_2 = ''' 

-- присоединяем к таблице с вакансиями названия городов
-- выводим название вакансии, название города, опыт работы, 
-- занятость и расписание 
select 
    v.name  vac_name, 
    a.name  city,
    e.name  employer,
    coalesce(salary_from, 0) salary_from,
    coalesce(salary_to, 0) salary_to     
from 
    vacancies v 
    join areas a 
    on v.area_id=a.id
    
    join employers e 
    on v.employer_id=e.id

where 

-- выбираем данные, связанные с DS и ML
     (v.name ilike 'ml%'or
     v.name ilike '% ml%'or
     v.name ilike '%-ml%'or
     v.name ilike '%/ml%'or
     v.name ilike '%(ml%'or
     
        
     v.name ilike '%data scientist%' or 
     v.name ilike '%data science%' or 
     v.name ilike '%исследователь данных%' or
     v.name ilike '%machine learning%' or 
     v.name ilike '%машинн%обучен%')
     
     AND
     
     (salary_from is not null or salary_to is not null)
'''


In [ ]:
# создаем функцию Avg_slry для поиска средней ЗП
def Avg_slry(data_frame):
    
    """ 
        Эта функция расчитывает среднее по ЗП, если оба 
        значения не равны нулю. Если значение salary_from == 0, 
        принимает значение salary_to. И наоборот, Если значение 
        salary_to == 0, принимает значение salary_from. 
        NB: Перед использованием проверить сброшены ли индексы

    Args:
        data_frame (pd.dataframe): Датафрейм, в котором есть колонки 
        salary_from (ЗП от) и salary_to (ЗП до)

    Returns:
        pd.dataframe: Датафрейм с дополнительным столбцом 'Avg_salary' 
        (среднее значение по ЗП)
    """
    
    for i in range (0, len(data_frame)):
    
        if data_frame.loc[i, 'salary_from'] == 0 or \
            data_frame.loc[i, 'salary_to'] == 0:
                
            data_frame.loc[i, 'Avg_salary'] = \
                (data_frame.loc[i, 'salary_to'] + \
                    data_frame.loc[i, 'salary_from'])
            
               
        else: data_frame.loc[i, 'Avg_salary'] = \
            (data_frame.loc[i, 'salary_to'] + data_frame.loc[i, 'salary_from']) / 2
            
    return data_frame
            

In [711]:
# Записываем результат запроса в переменную и оставляем только те данные, 
# которые соответсвуют городам-миллионникам 
add_df_2 = pd.read_sql_query(add_query_2, connection)

mask = add_df_2.loc[:, 'city'].apply(lambda x: x in city_names)
add_df_2 = add_df_2[mask].reset_index(drop=True)


# добавляем столбец со средней ЗП и группируем по городам и работодателям
add_df_2 = Avg_slry(add_df_2)

CityEmpl_data = add_df_2.groupby(['city', 'employer'],\
    as_index=False)['Avg_salary'].mean()

CityEmpl_data = CityEmpl_data.sort_values('Avg_salary', ascending=False)


# Выводм данные по ЗП в зависимости от работадателя в виде барплота
Slry_by_CityEmpl = px.bar(
    data_frame= CityEmpl_data,
    x = 'employer',
    y = 'Avg_salary',
    color = 'city', 
    width = 1400,
    height = 900,
    title = 'Зависимость средней зарплаты от работодателя<br>\
(данные представлены по городам-миллионникам)'
)


Slry_by_CityEmpl.update_layout(yaxis = dict(title = 'Средняя зарплата'),
                               xaxis = dict(title = ''),
                               legend = dict(title = ''))
                               
Slry_by_CityEmpl.show()
Slry_by_CityEmpl.write_html(path + '/HTMLs/Slry_by_CityEmpl.html')
Slry_by_CityEmpl.write_image(path + '/PNGs/Slry_by_CityEmpl.png', scale=6.5)


* Наибольшее количество компаний, которые ищут сотрудников в сфере DS и ML расположено в *Москве*. На втором месте *Санкт-Петербург*. <br>
В пятерку лидеров также входят *Новосибирск*, *Екатеринбург* и *Пермь*. Наибольшую ЗП в **450к** предлагает компания "Итон" (Москва). Наименьшую - компания "Ранкс" (Казань).

* Зарплату выше **150к** рублей предлагают преимущественно работодатели в *Москве*, однако отдельные компании предлагают аналогичную зарплату в *СпБ*, *НСК* и *ЕКБ*.
* В *Перми*, *Уфе*, *Ростове-на-Дону* и *Казани* не предлагают зарплату выше **100к**.

In [ ]:
# Записываем результат запроса в переменную и оставляем только те данные, 
# которые соответсвуют городам-миллионникам 
add_df_3 = pd.read_sql_query(add_query_3, connection)
add_df_3 = Avg_slry(add_df_3)




Slry_by_IndEmpl = add_df_3.groupby(['employer', 'industry'], \
    as_index=False)['Avg_salary'].mean().sort_values(['employer', 'Avg_salary'])
display(Slry_by_IndEmpl)




employer  \
0                                  Andersen   
1                                  Andersen   
2                                AppQuantum   
3                                AppQuantum   
4                                AppQuantum   
..                                      ...   
72                         Университет ИТМО   
73                         Университет ИТМО   
74  Центр хранения и анализа больших данных   
75  Центр хранения и анализа больших данных   
76  Центр хранения и анализа больших данных   

                                             industry     Avg_salary  
0                 Разработка программного обеспечения  268862.666667  
1   Системная интеграция,  автоматизации технологи...  268862.666667  
2                           Издательская деятельность  350000.000000  
3   Интернет-компания (поисковики, платежные систе...  350000.000000  
4   Маркетинговые, рекламные, BTL, дизайнерские, E...  350000.000000  
..                                                ...            ...  
72                        Государственные организации  115000.000000  
73  Научно-исследовательская, научная, академическ...  115000.000000  
74                              Консалтинговые услуги  170000.000000  
75                Разработка программного обеспечения  170000.000000  
76  Системная интеграция,  автоматизации технологи...  170000.000000  

[77 rows x 3 columns]

In [ ]:
# выводим полученные данные в виде тепловой карты
Slry_by_IndEmpl_imsh = px.bar(
    data_frame=Slry_by_IndEmpl,
    x = 'industry',
    y = 'Avg_salary',
    color = 'employer', 
    
    title = 'Зависимость средней зарплаты от работодателя<br>\
(данные представлены по городам-миллионникам)'
)
    

Slry_by_IndEmpl_imsh.show()
Slry_by_IndEmpl_imsh.write_html(path + '/HTMLs/Slry_by_IndEmpl_imsh.html')
Slry_by_IndEmpl_imsh.write_image(path + '/PNGs/Slry_by_IndEmpl_imsh.png', scale=6.5)

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования